In [10]:
from tensorflow.keras.layers import Input, Dropout, GlobalAveragePooling2D, Dense, LeakyReLU, Conv2D, MaxPooling2D,\
      Flatten, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model
from timeit import default_timer as timer
from tensorflow.keras.applications.inception_v3 import InceptionV3
import numpy as np
from sklearn.metrics import classification_report
import const
import os


In [13]:
# diverted_images_train_path = os.path.join(const.DATASET_CLEAN, "train")
# diverted_images_valid_path = os.path.join(const.DATASET_CLEAN, "valid")
# diverted_images_test_path = os.path.join(const.DATASET_CLEAN, "test")

# Remplacez 'chemin_vers_le_nouveau_repertoire' par le chemin du nouveau répertoire
const.DATASET = "..\\data\\dataset_reduit"

diverted_images_train_path = os.path.join(const.DATASET, "train")
diverted_images_valid_path = os.path.join(const.DATASET, "valid")
diverted_images_test_path = os.path.join(const.DATASET, "test")


for path in [diverted_images_train_path, diverted_images_valid_path, diverted_images_test_path]:
    if not os.path.isdir(path):
        print(f"Le dossier {path} n'existe pas. Veuillez vérifier le chemin.")

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

In [13]:
#0.64-0.62
# meilleur modèle avec augmentation des données
batch_size = 16
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), 
                                                    batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), 
                                                    batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), 
                                                  batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 980 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Epoch 1/20
61/61 [==============================] - 70s 1s/step - loss: 2.3916 - acc: 0.2562 - mean_absolute_error: 0.1585 - val_loss: 3.6448 - val_acc: 0.1010 - val_mean_absolute_error: 0.1775 - lr: 0.0010
Epoch 2/20
61/61 [==============================] - 70s 1s/step - loss: 1.9951 - acc: 0.3485 - mean_absolute_error: 0.1461 - val_loss: 4.2898 - val_acc: 0.1394 - val_mean_absolute_error: 0.1721 - lr: 0.0010
Epoch 3/20
61/61 [==============================] - ETA: 0s - loss: 2.0624 - acc: 0.3185 - mean_absolute_error: 0.1473
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
61/61 [==============================] - 72s 1s/step - loss: 2.0624 - acc: 0.3185 - mean_absolute_error: 0.1473 - val_loss: 3.7046 - val_acc: 0.1490 - val_mean_absolute_error: 0.1732 - lr: 0.0010
Epoch 4/20
61/61 [==============================] - 72s 1s/step - lo

In [14]:
#0.56-0.60
#modèle sequentiel avec Batchnormalization


def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Epoch 1/20
61/61 [==============================] - 71s 1s/step - loss: 2.6808 - acc: 0.1992 - mean_absolute_error: 0.1653 - val_loss: 2.8446 - val_acc: 0.1394 - val_mean_absolute_error: 0.1777 - lr: 0.0010
Epoch 2/20
61/61 [==============================] - 71s 1s/step - loss: 2.2090 - acc: 0.3143 - mean_absolute_error: 0.1473 - val_loss: 4.3691 - val_acc: 0.1058 - val_mean_absolute_error: 0.1755 - lr: 0.0010
Epoch 3/20
61/61 [==============================] - ETA: 0s - loss: 2.0728 - acc: 0.3299 - mean_absolute_error: 0.1445
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
61/61 [==============================] - 73s 1s/step - loss: 2.0728 - acc: 0.3299 - mean_absolute_error: 0.1445 - val_loss: 5.4859 - val_acc: 0.1058 - val_mean_absolute_error: 0.1783 - lr: 0.0010
Epoch 4/20
61/61 [==============================] - 74s 1s/step - loss: 1.7814 - acc: 0.4077 - mean_absolute_error: 0.1335 - val_loss: 4.8326 - val_acc: 0.1154 - val_mean_absolute_error: 0.1760 

In [15]:
#0.59-0.56
def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Epoch 1/20
61/61 [==============================] - 71s 1s/step - loss: 2.4231 - acc: 0.2490 - mean_absolute_error: 0.1611 - val_loss: 2.9219 - val_acc: 0.1587 - val_mean_absolute_error: 0.1778 - lr: 0.0010
Epoch 2/20
61/61 [==============================] - 70s 1s/step - loss: 1.8163 - acc: 0.4015 - mean_absolute_error: 0.1378 - val_loss: 3.5625 - val_acc: 0.1106 - val_mean_absolute_error: 0.1763 - lr: 0.0010
Epoch 3/20
61/61 [==============================] - ETA: 0s - loss: 1.6009 - acc: 0.4388 - mean_absolute_error: 0.1296
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
61/61 [==============================] - 71s 1s/step - loss: 1.6009 - acc: 0.4388 - mean_absolute_error: 0.1296 - val_loss: 6.0837 - val_acc: 0.1442 - val_mean_absolute_error: 0.1725 - lr: 0.0010
Epoch 4/20
61/61 [==============================] - 75s 1s/step - loss: 1.9147 - acc: 0.3745 - mean_absolute_error: 0.1404 - val_loss: 5.1418 - val_acc: 0.1250 - val_mean_absolute_error: 0.1757 

In [16]:
#augmentation de la complexité du modèle, avec augmentation de données
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

def build_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])

Found 980 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Epoch 1/20
61/61 [==============================] - 135s 2s/step - loss: 2.7395 - acc: 0.2033 - mean_absolute_error: 0.1658 - val_loss: 6.0258 - val_acc: 0.1923 - val_mean_absolute_error: 0.1624 - lr: 0.0010
Epoch 2/20
61/61 [==============================] - 156s 3s/step - loss: 2.4348 - acc: 0.2676 - mean_absolute_error: 0.1556 - val_loss: 4.3190 - val_acc: 0.1442 - val_mean_absolute_error: 0.1723 - lr: 0.0010
Epoch 3/20
61/61 [==============================] - 148s 2s/step - loss: 2.0776 - acc: 0.3278 - mean_absolute_error: 0.1454 - val_loss: 6.0378 - val_acc: 0.1058 - val_mean_absolute_error: 0.1781 - lr: 0.0010
Epoch 4/20
61/61 [==============================] - 144s 2s/step - loss: 2.1367 - acc: 0.3133 - mean_absolute_error: 0.1448 - val_loss: 4.2688 - val_acc: 0.1490 - val_mean_absolute_error: 0.1725 - lr: 0.0010
Epoch 5/20
61/61 [========================

In [27]:
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

def build_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))  

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.35))  

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.45))  

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.55)) 

    model.add(Flatten())
    model.add(Dense(512, activation='relu')) 
    model.add(BatchNormalization())
    model.add(Dropout(0.6))  
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 980 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Epoch 1/20
61/61 [==============================] - 160s 3s/step - loss: 3.0241 - acc: 0.2106 - mean_absolute_error: 0.1641 - val_loss: 13.9857 - val_acc: 0.1010 - val_mean_absolute_error: 0.1797 - lr: 0.0010
Epoch 2/20
61/61 [==============================] - 155s 3s/step - loss: 2.9061 - acc: 0.2573 - mean_absolute_error: 0.1554 - val_loss: 13.0044 - val_acc: 0.1010 - val_mean_absolute_error: 0.1798 - lr: 0.0010
Epoch 3/20
61/61 [==============================] - 158s 3s/step - loss: 2.5673 - acc: 0.2936 - mean_absolute_error: 0.1484 - val_loss: 4.7631 - val_acc: 0.1106 - val_mean_absolute_error: 0.1780 - lr: 0.0010
Epoch 4/20
61/61 [==============================] - 147s 2s/step - loss: 2.2306 - acc: 0.3568 - mean_absolute_error: 0.1401 - val_loss: 9.0159 - val_acc: 0.1010 - val_mean_absolute_error: 0.1802 - lr: 0.0010
Epoch 5/20
61/61 [======================

In [26]:
batch_size = 16

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

train_generator = ImageDataGenerator().flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

num_classes = train_generator.num_classes

def build_model():
    input_layer = Input(shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=predictions)
    return model

model = build_model()
#model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 980 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Epoch 1/20
61/61 [==============================] - 2s 35ms/step - loss: 2.9491 - acc: 0.2085 - mean_absolute_error: 0.1649 - val_loss: 2.3824 - val_acc: 0.2692 - val_mean_absolute_error: 0.1567 - lr: 0.0010
Epoch 2/20
61/61 [==============================] - 2s 33ms/step - loss: 2.0964 - acc: 0.2562 - mean_absolute_error: 0.1615 - val_loss: 2.0288 - val_acc: 0.2788 - val_mean_absolute_error: 0.1569 - lr: 0.0010
Epoch 3/20
61/61 [==============================] - 2s 33ms/step - loss: 1.9463 - acc: 0.2842 - mean_absolute_error: 0.1580 - val_loss: 1.9205 - val_acc: 0.2837 - val_mean_absolute_error: 0.1526 - lr: 0.0010
Epoch 4/20
61/61 [==============================] - 2s 33ms/step - loss: 1.8967 - acc: 0.2905 - mean_absolute_error: 0.1554 - val_loss: 1.8990 - val_acc: 0.3125 - val_mean_absolute_error: 0.1496 - lr: 0.0010
Epoch 5/20
61/61 [========================

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras import regularizers
import os
from timeit import default_timer as timer

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

# Chemins vers les dossiers de données
const.DATASET = "..\\data\\dataset_reduit"
diverted_images_train_path = os.path.join(const.DATASET, "train")
diverted_images_valid_path = os.path.join(const.DATASET, "valid")
diverted_images_test_path = os.path.join(const.DATASET, "test")

# Vérifiez que les dossiers existent
for path in [diverted_images_train_path, diverted_images_valid_path, diverted_images_test_path]:
    if not os.path.isdir(path):
        print(f"Le dossier {path} n'existe pas. Veuillez vérifier le chemin.")

# Paramètres du modèle
batch_size = 16

# Préparation des générateurs de données
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Création des générateurs de données
train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

# Affichage du nombre de classes
print(f"Nombre de classes dans les données d'entraînement : {train_generator.num_classes}")
print(f"Nombre de classes dans les données de validation : {valid_generator.num_classes}")
print(f"Nombre de classes dans les données de test : {test_generator.num_classes}")

# Construction du modèle
def build_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3), kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # Ajoutez plus de couches ici si nécessaire

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(train_generator.num_classes, activation='softmax'))

    return model

# Compilation du modèle
model = build_model()
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=50, 
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

# Évaluation du modèle
test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 980 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Nombre de classes dans les données d'entraînement : 10
Nombre de classes dans les données de validation : 10
Nombre de classes dans les données de test : 10
Epoch 1/50
61/61 [==============================] - 62s 1s/step - loss: 3.5952 - acc: 0.2386 - mean_absolute_error: 0.1569 - val_loss: 3.1557 - val_acc: 0.1010 - val_mean_absolute_error: 0.1782 - lr: 1.0000e-04
Epoch 2/50
61/61 [==============================] - 62s 1s/step - loss: 2.7649 - acc: 0.3724 - mean_absolute_error: 0.1360 - val_loss: 3.2807 - val_acc: 0.1010 - val_mean_absolute_error: 0.1768 - lr: 1.0000e-04
Epoch 3/50
61/61 [==============================] - ETA: 0s - loss: 2.6476 - acc: 0.3869 - mean_absolute_error: 0.1325
Epoch 3: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
61/61 [==============================] - 65s 1s/step - loss: 2.6476 - acc: 0.3869 - mean_absolute_er

In [14]:
#augmentation de la complexité du modèle, avec augmentation de données
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size, class_mode='categorical')

num_classes = train_generator.num_classes

reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

def build_model():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = build_model()

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])

Found 980 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Epoch 1/20
61/61 [==============================] - 137s 2s/step - loss: 2.7480 - acc: 0.1992 - mean_absolute_error: 0.1651 - val_loss: 4.3200 - val_acc: 0.1010 - val_mean_absolute_error: 0.1799 - lr: 1.0000e-04
Epoch 2/20
61/61 [==============================] - 177s 3s/step - loss: 1.9863 - acc: 0.3548 - mean_absolute_error: 0.1422 - val_loss: 5.4800 - val_acc: 0.0962 - val_mean_absolute_error: 0.1806 - lr: 1.0000e-04
Epoch 3/20
61/61 [==============================] - ETA: 0s - loss: 1.8057 - acc: 0.4066 - mean_absolute_error: 0.1330
Epoch 3: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
61/61 [==============================] - 170s 3s/step - loss: 1.8057 - acc: 0.4066 - mean_absolute_error: 0.1330 - val_loss: 4.3526 - val_acc: 0.0962 - val_mean_absolute_error: 0.1799 - lr: 1.0000e-04
Epoch 4/20
61/61 [==============================] - 17

KeyboardInterrupt: 

In [15]:
#conversion en fonctionnel du meilleur modèle obtenu plus haut
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

def build_functional_model():
    inputs = Input(shape=(224, 224, 3))

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.5)(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

model = build_functional_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Epoch 1/20
61/61 [==============================] - 184s 3s/step - loss: 2.5278 - acc: 0.2479 - mean_absolute_error: 0.1584 - val_loss: 7.9708 - val_acc: 0.1010 - val_mean_absolute_error: 0.1794 - lr: 0.0010
Epoch 2/20
61/61 [==============================] - 191s 3s/step - loss: 2.3338 - acc: 0.2873 - mean_absolute_error: 0.1486 - val_loss: 52.5603 - val_acc: 0.1010 - val_mean_absolute_error: 0.1798 - lr: 0.0010
Epoch 3/20
61/61 [==============================] - ETA: 0s - loss: 2.2728 - acc: 0.3122 - mean_absolute_error: 0.1477
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
61/61 [==============================] - 172s 3s/step - loss: 2.2728 - acc: 0.3122 - mean_absolute_error: 0.1477 - val_loss: 18.3670 - val_acc: 0.1010 - val_mean_absolute_error: 0.1796 - lr: 0.0010
Epoch 4/20
61/61 [==============================] - 177s 3s/step - loss: 2.1578 - acc: 0.3392 - mean_absolute_error: 0.1439 - val_loss: 7.7942 - val_acc: 0.1154 - val_mean_absolute_error: 0

In [19]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    diverted_images_train_path, 
    target_size=(224, 224), 
    batch_size=batch_size, 
    class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(
    diverted_images_valid_path, 
    target_size=(224, 224), 
    batch_size=batch_size, 
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    diverted_images_test_path, 
    target_size=(224, 224), 
    batch_size=batch_size, 
    class_mode='categorical')


def build_functional_model():
    inputs = Input(shape=(224, 224, 3))

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.5)(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

model = build_functional_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=40,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Found 980 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Found 210 images belonging to 10 classes.
Epoch 1/40
61/61 [==============================] - 137s 2s/step - loss: 2.8400 - acc: 0.2075 - mean_absolute_error: 0.1680 - val_loss: 7.6253 - val_acc: 0.1010 - val_mean_absolute_error: 0.1792 - lr: 0.0010
Epoch 2/40
61/61 [==============================] - 162s 3s/step - loss: 2.4322 - acc: 0.2313 - mean_absolute_error: 0.1609 - val_loss: 21.5498 - val_acc: 0.1010 - val_mean_absolute_error: 0.1798 - lr: 0.0010
Epoch 3/40
61/61 [==============================] - 154s 3s/step - loss: 2.2978 - acc: 0.2790 - mean_absolute_error: 0.1547 - val_loss: 7.2761 - val_acc: 0.1010 - val_mean_absolute_error: 0.1797 - lr: 0.0010
Epoch 4/40
61/61 [==============================] - 161s 3s/step - loss: 2.2806 - acc: 0.3019 - mean_absolute_error: 0.1515 - val_loss: 10.0905 - val_acc: 0.1058 - val_mean_absolute_error: 0.1769 - lr: 0.0010
Epoch 5/40
61/61 [======================

In [20]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

def build_functional_model():
    inputs = Input(shape=(224, 224, 3))

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)

    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)

    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.5)(x)

    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

model = build_functional_model()

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=40,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


Epoch 1/40
61/61 [==============================] - 138s 2s/step - loss: 2.7765 - acc: 0.1878 - mean_absolute_error: 0.1654 - val_loss: 9.5268 - val_acc: 0.1058 - val_mean_absolute_error: 0.1782 - lr: 0.0010
Epoch 2/40
61/61 [==============================] - 155s 3s/step - loss: 2.6790 - acc: 0.2127 - mean_absolute_error: 0.1627 - val_loss: 7.0646 - val_acc: 0.0865 - val_mean_absolute_error: 0.1808 - lr: 0.0010
Epoch 3/40
61/61 [==============================] - 149s 2s/step - loss: 2.4837 - acc: 0.2479 - mean_absolute_error: 0.1564 - val_loss: 2.9813 - val_acc: 0.1779 - val_mean_absolute_error: 0.1638 - lr: 0.0010
Epoch 4/40
61/61 [==============================] - 145s 2s/step - loss: 2.2237 - acc: 0.2790 - mean_absolute_error: 0.1516 - val_loss: 6.9302 - val_acc: 0.1010 - val_mean_absolute_error: 0.1771 - lr: 0.0010
Epoch 5/40
61/61 [==============================] - ETA: 0s - loss: 1.9755 - acc: 0.3537 - mean_absolute_error: 0.1415
Epoch 5: ReduceLROnPlateau reducing learning rate